In [7]:
import pandas as pd
import shutil
import tempfile
import xlwings as xw
import numpy as np
import glob
import re
from datetime import datetime
import os

from tkinter import *
from tkinter import ttk
from tkinter import filedialog

In [16]:
# Allgemeine Funktionen
def return_error_rows_as_string(error_row):
    error_str = ''
    for e in error_row:
        error_str = error_str + str(e) + ', '
    error_str = error_str[:-2]    
    # Split the string by commas and convert each element to an integer
    numbers_list = [int(num.strip()) for num in error_str.split(',')]
    # Sort the list in ascending order
    numbers_list.sort()
    # Initialize variables
    result = []
    start = numbers_list[0]
    end = numbers_list[0]
    # Iterate over the list and find consecutive numbers
    for i in range(1, len(numbers_list)):
        if numbers_list[i] == end + 1:
            end = numbers_list[i]
        else:
            # Add the range to the result
            if start == end:
                result.append(str(start))
            else:
                result.append(f'{start}-{end}')
            start = numbers_list[i]
            end = numbers_list[i]
    # Add the last range to the result
    if start == end:
        result.append(str(start))
    else:
        result.append(f'{start}-{end}')
    # Join the ranges with commas
    error_str = ', '.join(result)    
    return(error_str)



def zellen_bunt_malen(error_str, column_name, ws, color):
    # get column index from column name
    column_index = vorlage.columns.get_loc(column_name)
    column_letter = xw.utils.col_name(column_index + 1)
    # cell coordinates
    error_row_list = error_str.split(',')
    for error_row in error_row_list:
        if '-' in error_row:
            first_row = error_row.split('-')[0].strip()
            last_row = error_row.split('-')[1].strip()
            cell_coord = column_letter + first_row + ':' + column_letter + last_row
        else:
            cell_coord = column_letter + error_row.strip()
        cell = ws.range(cell_coord)
        cell.color = color
        
        

def open_dm_file(name, path, sheetname):
    """ Öffnet eine der Dateien des DMs.
        Dafür werden der Name der Datei, Pfad und sheet name als Argumente übergeben.
    """    
    files = glob.glob(path + '\*.xlsx*')
    for i in range(len(files)):
        if name in files[i]:
            path = files[i]
    try:
        excel = pd.read_excel(path, sheet_name = sheetname)
    # falls Datei geöffnet ist:
    except:
        if '~$' in path:
            path = path.replace('~$', '')
        # Generate a temporary directory to store the copied file
        temp_dir = tempfile.mkdtemp()        
        # Generate a temporary file name with a .xlsx extension in the temporary directory
        temp_file_path = tempfile.mktemp(suffix='.xlsx', dir=temp_dir)        
        # Copy the open Excel file to the temporary location
        shutil.copy2(path, temp_file_path)
        excel = pd.read_excel(temp_file_path, sheet_name = sheetname)    
        shutil.rmtree(temp_dir)
    return(excel)



def open_closed_or_opened_file(path):
    """ Mit der Funktion wird zuerst versucht, die Excel-Datei mit pandas zu öffnen.
        Falls die Datei bereits von einem User geöffnet ist, wird eine Kopie erstellt.
    """
    try:
        df = pd.read_excel(path)
    except:
        if '~$' in path:
            path = path.replace('~$', '')
        # Generate a temporary directory to store the copied file
        temp_dir = tempfile.mkdtemp()        
        # Generate a temporary file name with a .xlsx extension in the temporary directory
        temp_file_path = tempfile.mktemp(suffix='.xlsx', dir=temp_dir)        
        # Copy the open Excel file to the temporary location
        shutil.copy2(path, temp_file_path)        
        df = pd.read_excel(temp_file_path)    
        shutil.rmtree(temp_dir)    
    return(df)

In [17]:
# Check-Funktionen

# Check 1: Master ID
def check_masterid(vorlage, ws, error_count, error_count_total):
    master_id_pattern = r'I\d{9}\b'
    error_master_id_row = []
    for i in range(len(vorlage['master_id'])):
        match = re.search(master_id_pattern, str(vorlage['master_id'][i]))
        if match:
            continue
        else:
            if str(vorlage['master_id'][i]) == 'nan':
                continue
            else:
                error_master_id_row.append(i+2)
    if error_master_id_row != []:
        error_str = return_error_rows_as_string(error_master_id_row)
        text_master = Label(frame, text = "Fehler in Spalte 'master_id' Zeile(n) {}".format(error_str),
                            bg = '#eeeee4', font=('Ink free',11))
        text_master.pack()
        # Zelle in Excel-Datei rot färben
        zellen_bunt_malen(error_str, 'master_id', ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1
        

# Check 2: Sample ID
def check_sampleid(cols, cols_l, vorlage, ws, error_count, error_count_total):
    sample_id_pattern = r'I\d{12}\b'
    col_i = cols_l.index('sample_id')
    error_sample_id_row = []
    for i in range(len(vorlage[cols[col_i]])):
        match = re.search(sample_id_pattern, str(vorlage[cols[col_i]][i]))
        if match:
            continue
        else:
            if str(vorlage[cols[col_i]][i]) == 'nan':
                continue
            else:
                error_sample_id_row.append(i+2)
    if error_sample_id_row != []:
        error_str = return_error_rows_as_string(error_sample_id_row)
        text_sample = Label(frame, text = "Fehler in Spalte 'Sample_id' Zeile(n) {}".format(error_str),
                                bg = '#eeeee4', font=('Ink free',11))
        text_sample.pack()
        # Zelle in Excel-Datei rot färben
        zellen_bunt_malen(error_str, cols[col_i], ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1        

# Check 3: Pat ID
def check_patid(vorlage, ws, error_count, error_count_total):
    error_pat_id_row = []
    # Zuerst wird geprüft, ob die gesamte Spalte im Format np.int64 ist
    if vorlage['pat_id'].dtype != np.int64:
        if vorlage['pat_id'].dtype == 'object':
            for i in range(len(vorlage['pat_id'])):
                if not str(vorlage['pat_id'][i]).isdigit():
                    error_pat_id_row.append(i+2)
    if error_pat_id_row != []:
        error_str = return_error_rows_as_string(error_pat_id_row)
        text_pat = Label(frame, text = "(Format-)Fehler in Spalte 'pat_id' Zeile(n) {}".format(error_str),
                        bg = '#eeeee4', font=('Ink free',11))
        text_pat.pack()
        # Zelle in Excel-Datei rot färben
        zellen_bunt_malen(error_str, 'pat_id', ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1
        
# Check 4: Sample und Master ID
def check_sample_master(vorlage, ws, error_count, error_count_total):
    error_sample_master = []
    for i in range(len(vorlage['Sample_id'])):
        if str(vorlage['Sample_id'][i])[:-3] != str(vorlage['master_id'][i]):
            error_sample_master.append(i+2)
    if error_sample_master != []:
        error_str = return_error_rows_as_string(error_sample_master)
        text_pat = Label(frame, text = "'Sample id' stimmt nicht mit 'master_id' überein in Zeile(n) {}".format(error_str),
                        bg = '#eeeee4', font=('Ink free',11))
        text_pat.pack()
        # Zelle in Excel-Datei rot färben
        zellen_bunt_malen(error_str, 'Sample_id', ws, (220, 20, 60))
        zellen_bunt_malen(error_str, 'master_id', ws, (220, 20, 60))
        error_count_total[0] += 1
        error_count[0] += 1
        
# Check 5: monovalent lists
def check_monovalent(felder_f, ws, error_count, error_count_total):
    for i in range(len(felder_f)):
        field_error_row = []
        if 'Monovalent list' in felder_f['Feldart'][i]:
            check_col = felder_f['Feldcode'][i]
            # falls Auswahlmöglichkeiten in einer weiteren Excel-Datei hinterlegt sind,
            # wird der Pfad angegeben
            possible_choices = felder_f['Auswahlmöglichkeiten bei Listen'][i]
            if 'O:' in possible_choices:
                if '\n' in possible_choices:
                    possible_choices = possible_choices.replace('\n', '')
                path = possible_choices.split('siehe ')[-1]
                if '-->' in path:
                    filename = path.split('-->')[1].strip()
                    path = path.split('-->')[0].strip()
                else:
                    filename = path.split('\\')[-1].split('_')[0]
                    path = '\\'.join(path.split('\\')[:-1])
                files = glob.glob(path+'\*.xlsx')
                for file in files:
                    if (filename in file) | (' '.join(filename.split('_')) in file):
                        path = file                
                df_choices = open_closed_or_opened_file(path)
                # erste Spalte ablesen und als Liste abspeichern
                possible_choices = df_choices.iloc[:,0].tolist()
            else:
                if '\n' in possible_choices:
                    possible_choices = possible_choices.split('\n')
                elif ',' in possible_choices:
                    possible_choices = possible_choices.split(',')
            # muss ggf. in List umgewandelt werden
            if not isinstance(possible_choices, list):
                possible_choices = [possible_choices]    
            for j in range(len(possible_choices)):
                c = possible_choices[j].split('=')[0]
                try:
                    c = int(c)
                    possible_choices[j] = possible_choices[j].split('=')[0]
                except:
                    try:
                        possible_choices[j] = possible_choices[j].split('=')[1]
                    except:
                        possible_choices[j] = '00'

            possible_choices = [str(int(c)) for c in possible_choices]
            # Ablesen des Eintrags in der zu importierenden Datei (vorlage)
            check_entry = list(vorlage[check_col])
            for j in range(len(check_entry)):                      
                if str(check_entry[j])=='nan':
                    continue
                else:
                    try:
                        if str(int(check_entry[j])) not in possible_choices:
                            field_error_row.append(j+2)
                        elif ('.' in str(check_entry[j])) & (str(check_entry[j]).split('.')[-1]!='0'):
                            field_error_row.append(j+2)
                    except:
                        field_error_row.append(j+2)
        # Falls Fehler in eins der Felder, dann erscheint Fehlermeldung
        if field_error_row != []:
            error_str = return_error_rows_as_string(field_error_row)
            text_field = Label(frame, text = "Fehler in Spalte '{}' Zeile(n) {}".format(check_col, error_str),
                                bg = '#eeeee4', font=('Ink free',11))
            text_field.pack()
            # Zelle in Excel-Datei rot färben
            zellen_bunt_malen(error_str, check_col, ws, (220, 20, 60))
            error_count_total[0] += 1
            error_count[0] += 1
            
            
# Check 6: Datum-Spalte
def check_datum(felder_f, vorlage, ws, error_count, error_count_total):
    for i in range(len(felder_f)):
        date_error_row = []
        if 'Date' in felder_f['Feldart'][i]:
            date_col = felder_f['Feldcode'][i]
            for j in range(len(vorlage)):
                if str(vorlage[date_col][j]) != 'nan':
                    try:
                        date_string = str(vorlage[date_col][j]).split(' ')[0]
                        try:
                            date_object = datetime.strptime(date_string, "%d.%m.%Y").date()
                        except:
                            try:
                                date_object = datetime.strptime(date_string, "%Y-%m").date()
                            except:
                                try:
                                    date_object = datetime.strptime(date_string, "%Y-%m-%d").date()
                                except:
                                    date_error_row.append(j+2)
                    except:
                        date_error_row.append(j+2)
        if date_error_row != []:
            error_str = return_error_rows_as_string(date_error_row)
            text_date = Label(frame, text = "Fehler in Datumspalte '{}' Zeile(n) {}".format(date_col, error_str),
                              bg = '#eeeee4', font=('Ink free',11))
            text_date.pack()
            # Zelle in Excel-Datei rot färben
            zellen_bunt_malen(error_str, date_col, ws, (220, 20, 60))
            error_count_total[0] += 1
            error_count[0] += 1
            
            
# Check 7: One line text
def check_one_line_text(felder_f, vorlage, ws, error_count, error_count_total):
    for i in range(len(felder_f)):
        long_text_row = []
        if 'One line text' in felder_f['Feldart'][i]:
            check_col = felder_f['Feldcode'][i]
            for j in range(len(vorlage)):
                text = vorlage[check_col][j]
                if len(str(text)) > 120:
                    long_text_row.append(j+2)
        if long_text_row != []:
            error_str = return_error_rows_as_string(long_text_row)
            text_long = Label(frame, text = "Text wurde abgeschnitten in Spalte '{}' Zeile(n) {}".format(check_col, error_str),
                              bg = '#eeeee4', font=('Ink free',11))
            text_long.pack()
            # Zelle in Excel-Datei gelb färben
            zellen_bunt_malen(error_str, check_col, ws, (238, 232, 170))
            error_count_total[0] += 1
            error_count[0] += 1
            
            
# Check 8: Katalog-Felder
def check_katalog(felder_f, ws, error_count, error_count_total):
    for i in range(len(felder_f)):
        field_error_row = []
        if 'Katalog' in felder_f['Feldart'][i]:
            check_col = felder_f['Feldcode'][i]
            possible_choices = felder_f['Auswahlmöglichkeiten bei Listen'][i]
            if '\n' in possible_choices:
                possible_choices = possible_choices.replace('\n', '')
            if 'O:' in possible_choices:
                path = possible_choices.split('siehe ')[-1]
                if '-->' in path:
                    filename = path.split('-->')[1].strip()
                    path = path.split('-->')[0].strip()
                else:
                    filename = path.split('\\')[-1].split('_')[0]
                    path = '\\'.join(path.split('\\')[:-1])
                files = glob.glob(path+'\*.xlsx')
                for file in files:
                    if (filename in file) | (' '.join(filename.split('_')) in file):
                        path = file                
            df_choices = open_closed_or_opened_file(path)
            # erste Spalte ablesen und als Liste abspeichern
            possible_choices = df_choices.iloc[:,0].tolist()
            possible_choices = [str(int(c)) for c in possible_choices if str(c) != 'nan']
            # Ablesen des Eintrags in der zu importierenden Datei (vorlage)
            check_entry = list(vorlage[check_col])
            for j in range(len(check_entry)):
                if str(check_entry[j])=='nan':
                    continue
                else:
                    try:
                        if str(int(check_entry[j])) not in possible_choices:
                            field_error_row.append(j+2)
                        elif ('.' in str(check_entry[j])) & (str(check_entry[j]).split('.')[-1]!='0'):
                            field_error_row.append(j+2)
                    except:
                        field_error_row.append(j+2)
        # Falls Fehler in eins der Felder, dann erscheint Fehlermeldung
        if field_error_row != []:
            error_str = return_error_rows_as_string(field_error_row)
            text_kat = Label(frame, text = "Fehler in Spalte '{}' Zeile(n) {}".format(check_col, error_str),
                            bg = '#eeeee4', font=('Ink free',11))
            text_kat.pack()
            # Zelle in Excel-Datei rot färben
            zellen_bunt_malen(error_str, check_col, ws, (220, 20, 60))
            error_count_total[0] += 1
            error_count[0] += 1
            
            
# Check 9: Feldcode-Name
def check_feldcode(cols, felder_f, ws, error_count, error_count_total):
    for col in cols:
        if col not in list(felder_f['Feldcode']):
            text_feldcode = Label(frame, text = "Bitte Feldcode '{}' überprüfen".format(col),
                                  bg = '#eeeee4', font=('Ink free',11))
            text_feldcode.pack()
            error_count_total[0] += 1
            error_count[0] += 1
            
            
# Check 10: Pflichtfeld
def check_pflichtfeld(felder_f, ws, error_count, error_count_total):
    for i in range(len(felder_f)):
        pflichtfeld_row = []
        soll_row = []
        if (felder_f['Pflichtfeld'][i] == 'Pflichtfeld') | ('Pflichtfeld?' in felder_f['Pflichtfeld'][i]) | ('leeres Pflichtfeld' in felder_f['Pflichtfeld'][i]):
            check_col = felder_f['Feldcode'][i]
            for j in range(len(vorlage)):
                if str(vorlage[check_col][j]) == 'nan':
                    pflichtfeld_row.append(j+2)
            if pflichtfeld_row != []:
                error_str = return_error_rows_as_string(pflichtfeld_row)
                text_pflicht = Label(frame, text = "Pflichtfeld ist leer in Spalte '{}' Zeile(n) {}".format(check_col, error_str),
                                        bg = '#eeeee4', font=('Ink free',11))
                text_pflicht.pack()
                # Zelle in Excel-Datei orange färben
                zellen_bunt_malen(error_str, check_col, ws, (255, 127, 80))
                error_count_total[0] += 1
                error_count[0] += 1
        elif ('soll' in felder_f['Pflichtfeld'][i]) & ('Angabe' not in felder_f['Pflichtfeld'][i]):
            check_col = felder_f['Feldcode'][i]
            for j in range(len(vorlage)):
                if str(vorlage[check_col][j]) == 'nan':
                    soll_row.append(j+2)
            if soll_row != []:
                error_str = return_error_rows_as_string(soll_row)
                text_soll = Label(frame, text = "Bitte Feld füllen in Spalte '{}' Zeile(n) {}".format(check_col, error_str),
                                      bg = '#eeeee4', font=('Ink free',11))
                text_soll.pack()
                # Zelle in Excel-Datei orange färben
                zellen_bunt_malen(error_str, check_col, ws, (255, 127, 80))
                error_count_total[0] += 1
                error_count[0] += 1

In [18]:
def go_dodo():
    """ Überprüfen aller Excel-Dateien im ausgewählten Ordner
    """
    global error_title, root_m, vorlage, canvas, frame, error_title2
    try:
        error_title.grid_forget()
    except:
        pass
    try:
        error_title2.grid_forget()
    except:
        pass
    try:
        root_m.grid_forget()
    except:
        pass

    
    error_count_total = [0] # weil Listen im Gegensatz zu Integers mutable sind
    
    
    folder_path = filedialog.askdirectory(title = "Ordner auswählen")
    file_list = os.listdir(folder_path)
    
    ### Interface ###    
    root_m = Frame(root, width=700, height=150, bg = '#eeeee4', highlightbackground='#869287',
                   highlightthickness=2)
    root_m.grid(row=5, column=1, columnspan=100)
    # Create a Canvas to hold the Frame and Scrollbars
    canvas = Canvas(root_m, width=700, bg = '#eeeee4')
    canvas.pack(side="left", fill="both", expand=True)

    # Create vertical scrollbar and associate it with the Canvas
    v_scrollbar = Scrollbar(root_m, command=canvas.yview)
    v_scrollbar.pack(side="right", fill="y")
    canvas.configure(yscrollcommand=v_scrollbar.set)

    # Create horizontal scrollbar and associate it with the Canvas
    h_scrollbar = Scrollbar(root_m, command=canvas.xview, orient="horizontal")
    h_scrollbar.pack(side="bottom", fill="x")
    canvas.configure(xscrollcommand=h_scrollbar.set)

    # Create a Frame to hold the label widgets
    frame = Frame(canvas, bg='#eeeee4')
    canvas.create_window((0, 0), window=frame, anchor="nw")


    
    for i in range(len(file_list)):
        error_count = [0]
        if 'csv' in file_list[i]:
            vorlage = pd.read_csv(folder_path+'/'+file_list[i], encoding='latin_1', delimiter=';')
            file_name = file_list[i].split('.csv')[0]
        elif 'xlsx' in file_list[i]:
            vorlage = open_closed_or_opened_file(folder_path+'/'+file_list[i])
            file_name = file_list[i].split('.xlsx')[0]
        cols = list(vorlage.columns)       
        vorlage.to_excel('{}_Check.xlsx'.format(file_name), index=False)
        wb = xw.Book('{}_Check.xlsx'.format(file_name))
        ws = wb.sheets['Sheet1']
        
        filename_text = Label(frame, text = "{}".format(file_name),
                                   bg = '#eeeee4', font=('Ink free',11,'bold'))
        filename_text.pack()


        # Einlesen der Excel-Tabelle 'Aufbau und Felder'
        felder = open_dm_file('IDA_Aufbau und Felder', 'O:\Datenmanagement\IDA_in.vent Datenbank', 'Felderbezeichnung')
        felder.columns = felder.iloc[6, :]
        felder = felder.drop(felder.index[:7]).reset_index().drop('index', axis=1)
        # Filtern nach Pflichtfeld:
        # Zeilen rausnehmen, wenn Feld 'nie genutzt' oder 'derzeit nicht genutzt' wird
        felder_not_used = felder[felder['Pflichtfeld'].isin(['nie genutzt', 'derzeit nicht genutzt'])]
        felder = felder[~felder['Feldcode'].isin(felder_not_used['Feldcode'])].reset_index().drop('index', axis = 1)

        # Spalten der hochgeladenen Datei mit den Feldern abgleichen und ein Subset erstellen
        index_felder = []
        for col in cols:
            for i in range(len(felder)):
                if col == felder['Feldcode'][i]:
                    index_felder.append(i)
        felder_f = felder.iloc[index_felder].reset_index().drop('index', axis=1)


        
        # Format von pat_id, master_id und sample_id überprüfen       
        # Spalte master_id
        if 'master_id' in vorlage.columns:
            check_masterid(vorlage, ws, error_count, error_count_total)

        # Spalte sample_id
        cols = list(vorlage.columns)
        cols_l = [col.lower() for col in cols]
        if 'sample_id' in cols_l:
            check_sampleid(cols, cols_l, vorlage, ws, error_count, error_count_total)

        # Spalte pat_id
        if 'pat_id' in vorlage.columns:
            check_patid(vorlage, ws, error_count, error_count_total)

        # Check, ob master_id und sample_id identisch sind
        if ('Sample_id' in vorlage.columns) & ('master_id' in vorlage.columns):
            check_sample_master(vorlage, ws, error_count, error_count_total)
            
        # Überprüfen der Auswahlmöglichkeiten, wenn Feldart monovalent list ist
        check_monovalent(felder_f, ws, error_count, error_count_total)
        
        # Datum-Spalte überprüfen (dd.mm.yyyy)
        check_datum(felder_f, vorlage, ws, error_count, error_count_total)

        # 'One line text'-Felder überprüfen, sodass es eine Warnung gibt,
        # wenn Text ab dem 120. Zeichen abgeschnitten wird
        check_one_line_text(felder_f, vorlage, ws, error_count, error_count_total)

        # Katalog-Felder überprüfen
        check_katalog(felder_f, ws, error_count, error_count_total)

        # Feldcode-Namen überprüfen
        check_feldcode(cols, felder_f, ws, error_count, error_count_total)

        # Pflichtfelder füllen
        check_pflichtfeld(felder_f, ws, error_count, error_count_total)
                    
        error_file = Label(frame, text = 'Anzahl Errors für {}: '.format(file_name) + str(error_count[0]), bg = '#eeeee4', font=('Ink free',9,'bold'))
        error_file.pack()
        
        ##### User Interface #####
        error_title = Label(root, text = '\nError-Meldungen: ' + str(error_count_total[0]), bg = '#eeeee4', font=('Ink free',11,'bold'))
        error_title.grid(row=4, column=7, columnspan=15)
        ##########################

        wb.save()
        
    # Calculate the desired frame width based on the canvas width
    frame_width = canvas.winfo_reqwidth()  # Use canvas width
    # Set the width of the scrollable_frame and prevent resizing
    frame.grid_propagate(False)  # Prevent resizing
    frame.config(width=frame_width)  # Set the width
    # Configure Scrollbar to control scrolling
    frame.update_idletasks()
    canvas.config(scrollregion=canvas.bbox("all"))
        
        
        
def go_dodo_solo():
    """ Überprüfen einer einzelnen Excel-Datei
    """
    global error_title, root_m, vorlage, canvas, frame, error_title2
    try:
        error_title.grid_forget()
    except:
        pass
    try:
        error_title2.grid_forget()
    except:
        pass
    try:
        root_m.grid_forget()
    except:
        pass

        
    error_count = 0
    
    
    vorlage_path = filedialog.askopenfilename(title = "Datei auswählen")
    if 'csv' in vorlage_path:
        vorlage = pd.read_csv(vorlage_path, encoding='latin_1', delimiter=';')
    elif 'xlsx' in vorlage_path:
        vorlage = open_closed_or_opened_file(vorlage_path)
    cols = list(vorlage.columns)
    # neue Datei xxx_Check.xlsx erstellen, um Farbe von fehlerhaften Zellen zu ändern
    file_name = vorlage_path.split('/')[-1].split('.csv')[0]
    vorlage.to_excel('{}_Check.xlsx'.format(file_name), index=False)
    wb = xw.Book('{}_Check.xlsx'.format(file_name))
    ws = wb.sheets['Sheet1']

    
    ### Interface ###    
    root_m = Frame(root, width=700, height=150, bg = '#eeeee4', highlightbackground='#869287',
                   highlightthickness=2)
    root_m.grid(row=5, column=1, columnspan=100)
    
    # Create a Canvas to hold the Frame and Scrollbars
    canvas = Canvas(root_m, width=700, bg = '#eeeee4')
    canvas.pack(side="left", fill="both", expand=True)

    # Create vertical scrollbar and associate it with the Canvas
    v_scrollbar = Scrollbar(root_m, command=canvas.yview)
    v_scrollbar.pack(side="right", fill="y")
    canvas.configure(yscrollcommand=v_scrollbar.set)

    # Create horizontal scrollbar and associate it with the Canvas
    h_scrollbar = Scrollbar(root_m, command=canvas.xview, orient="horizontal")
    h_scrollbar.pack(side="bottom", fill="x")
    canvas.configure(xscrollcommand=h_scrollbar.set)

    # Create a Frame to hold the label widgets
    frame = Frame(canvas, bg='#eeeee4')
    canvas.create_window((0, 0), window=frame, anchor="nw")
    
    # Einlesen der Excel-Tabelle 'Aufbau und Felder'
    felder = open_dm_file('IDA_Aufbau und Felder', 'O:\Datenmanagement\IDA_in.vent Datenbank', 'Felderbezeichnung')
    felder.columns = felder.iloc[6, :]
    felder = felder.drop(felder.index[:7]).reset_index().drop('index', axis=1)
    # Filtern nach Pflichtfeld:
    # Zeilen rausnehmen, wenn Feld 'nie genutzt' oder 'derzeit nicht genutzt' wird
    felder_not_used = felder[felder['Pflichtfeld'].isin(['nie genutzt', 'derzeit nicht genutzt'])]
    felder = felder[~felder['Feldcode'].isin(felder_not_used['Feldcode'])].reset_index().drop('index', axis = 1)
    
    # Spalten der hochgeladenen Datei mit den Feldern abgleichen und ein Subset erstellen
    index_felder = []
    for col in cols:
        for i in range(len(felder)):
            if col == felder['Feldcode'][i]:
                index_felder.append(i)
    felder_f = felder.iloc[index_felder].reset_index().drop('index', axis=1)

    error_count = [0]
    error_count_total = [0]
    # Format von pat_id, master_id und sample_id überprüfen       
    # Spalte master_id
    if 'master_id' in vorlage.columns:
        check_masterid(vorlage, ws, error_count, error_count_total)

    # Spalte sample_id
    cols = list(vorlage.columns)
    cols_l = [col.lower() for col in cols]
    if 'sample_id' in cols_l:
        check_sampleid(cols, cols_l, vorlage, ws, error_count, error_count_total)

    # Spalte pat_id
    if 'pat_id' in vorlage.columns:
        check_patid(vorlage, ws, error_count, error_count_total)

    # Check, ob master_id und sample_id identisch sind
    if ('Sample_id' in vorlage.columns) & ('master_id' in vorlage.columns):
        check_sample_master(vorlage, ws, error_count, error_count_total)
            
    # Überprüfen der Auswahlmöglichkeiten, wenn Feldart monovalent list ist
    check_monovalent(felder_f, ws, error_count, error_count_total)
        
    # Datum-Spalte überprüfen (dd.mm.yyyy)
    check_datum(felder_f, vorlage, ws, error_count, error_count_total)

    # 'One line text'-Felder überprüfen, sodass es eine Warnung gibt,
    # wenn Text ab dem 120. Zeichen abgeschnitten wird
    check_one_line_text(felder_f, vorlage, ws, error_count, error_count_total)

    # Katalog-Felder überprüfen
    check_katalog(felder_f, ws, error_count, error_count_total)

    # Feldcode-Namen überprüfen
    check_feldcode(cols, felder_f, ws, error_count, error_count_total)

    # Pflichtfelder füllen
    check_pflichtfeld(felder_f, ws, error_count, error_count_total)
                
    ##### User Interface #####
    error_title2 = Label(root, text = '\nError-Meldungen: ' + str(error_count[0]), bg = '#eeeee4', font=('Ink free',11,'bold'))
    error_title2.grid(row=4, column=7, columnspan=15)
    ##########################
                
    wb.save()
    
    # Calculate the desired frame width based on the canvas width
    frame_width = canvas.winfo_reqwidth()  # Use canvas width
    # Set the width of the scrollable_frame and prevent resizing
    frame.grid_propagate(False)  # Prevent resizing
    frame.config(width=frame_width)  # Set the width
    # Configure Scrollbar to control scrolling
    frame.update_idletasks()
    canvas.config(scrollregion=canvas.bbox("all"))

In [19]:
# User Interface
# Basis
root = Tk()
root.title('Import-Check')
root.geometry('900x500')
root.iconbitmap('O:\Forschung & Entwicklung\Allgemein\Vorlagen\Abbildungen\Dodo\dodo_icon.ico')
root.config(bg='#eeeee4')

# alle Buttons auf Startseite
hi = Label(root, text = '\nÜberprüfen der Dateien vor dem Import in IDA                        \n',
           bg = '#eeeee4', font=('Ink free',15,'bold'))
hi.grid(row=1, column=1, columnspan=30)

browse_text = Label(root, text = 'Bitte wähle den Ordner aus  ', bg = '#eeeee4', font=('Ink free',12))
browse_text.grid(row=2, column=1, columnspan=5)
browse_button = Button(root, text='Browse', font=('Ink free',10,'bold'), bg='#869287',
                       command = go_dodo)
browse_button.grid(row=2, column=6)

browse_solo_text = Label(root, text = 'Oder wähle eine einzelne Datei aus ', bg = '#eeeee4', font=('Ink free',12))
browse_solo_text.grid(row=3, column=1, columnspan=5)
browse_solo_button = Button(root, text='Browse', font=('Ink free',10,'bold'), bg='#869287',
                       command = go_dodo_solo)
browse_solo_button.grid(row=3, column=6)


# Logo
from PIL import ImageTk, Image
frame_logo = Frame(root, width=1, height=1)
frame_logo.grid(row=1, column=0)

img = ImageTk.PhotoImage(Image.open("O:\Forschung & Entwicklung\Allgemein\Vorlagen\Abbildungen\Dodo\dodo-dancing_ohne Hintergrund_ohne Schatten.png").resize((70,70)), master = root)
label = Label(frame_logo, image = img, bg = '#eeeee4')
label.pack()

root.mainloop()